In [ ]:
import pandas as pd

In [ ]:
jbs= pd.read_csv('ICD-10CM order codes.csv')

In [ ]:
jbs

In [ ]:
jbs.drop_duplicates(inplace=True)
jbs

In [ ]:
jbs['Short Description']= jbs['Short Description'].apply( lambda x: x.lower())

In [ ]:
corpus= jbs['Short Description'].to_list()
#corpus

##### Text Cleaning

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [ ]:
corpus_embeddings = embedder.encode(corpus)

##### Clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans

#from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt
sum_squared = []
for i in list([700, 900, 1000, 1200, 1400, 1600]):
    kmeans = MiniBatchKMeans(n_clusters=i, init="k-means++", random_state=42, batch_size=10)
    kmeans.fit(corpus_embeddings)
    sum_squared.append(kmeans.inertia_)

In [ ]:
plt.plot(list([700, 900, 1000, 1200, 1400, 1600]), sum_squared)
plt.title("Elbow Graph")
plt.xlabel("Num Clusters")
plt.ylabel("Sum Squared")
plt.show

In [ ]:
k=1200
b=10
km= MiniBatchKMeans(n_clusters=k, batch_size=b )
km

In [ ]:
#pred=km.fit_predict(jbs)
model=km.fit(corpus_embeddings)
pred=model.predict(corpus_embeddings)

In [ ]:
corpus_embeddings

In [ ]:
import pickle

with open('clustering_kmeans_v4.3.pkl', 'wb') as fout:
    pickle.dump((model), fout)

In [ ]:
jbs['cluster']=pred
jbs['cluster'].value_counts()

##### Cluster Centroids

In [ ]:
jbs

In [ ]:
df=jbs.groupby('cluster')['Short Description'].value_counts()

In [ ]:
df

In [ ]:
jbs['cluster'].value_counts()

In [ ]:
# Encode a new sentence and predict its cluster
new_sentence = 'Patient is present with Typhiod fever'
embed = embedder.encode(new_sentence)
embed = embed.reshape(1, -1)
predicted_cluster = model.predict(embed)

In [ ]:
# Print the predicted cluster
print(predicted_cluster)

In [ ]:
from google.colab import files
files.download("data_with_clusteredv4.3.csv")
files.download("clustering_kmeans_v4.3.pkl")

In [ ]:
pip show scikit-learn